In [1]:
# import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# read in previously collected data
schools = pd.read_csv("Data/fbsSchoolData.csv") # season, team, id, mascot, abbrev, conference, division
playCalls = pd.read_csv("Data/fbsPlayCallData.csv") #season, team, conference, totalPlays, passingPlays, rushingPlays
winLoss = pd.read_csv("Data/fbsWinLossData.csv") #season, team, id, mascot, abbrev, conference, division, games, wins, losses, ties

# merge datasets together
merge_1 = pd.merge(schools, playCalls, on = ["season", "team"], how = "left")
merge_2 = pd.merge(merge_1, winLoss, on = ["season", "id"], how = "left")

# preview dataset
merge_2.head()

,season,team_x,id,mascot_x,abbreviation_x,conference_x,division_x,conference_y,totalPlays,passingPlays,rushingPlays,team_y,mascot_y,abbreviation_y,conference,division_y,games,wins,losses,ties
0,2004,Air Force,2005,Falcons,AFA,Mountain West,NaN,Mountain West,147.0,33.0,110.0,Air Force,Falcons,AFA,Mountain West,NaN,11,5,6,0
1,2004,Akron,2006,Zips,AKR,Mid-American,East,Mid-American,131.0,70.0,55.0,Akron,Zips,AKR,Mid-American,East,11,6,5,0
2,2004,Alabama,333,Crimson Tide,ALA,SEC,West,SEC,677.0,224.0,434.0,Alabama,Crimson Tide,ALA,SEC,West,11,6,5,0
3,2004,Arizona,12,Wildcats,ARIZ,Pac-10,NaN,Pac-12,681.0,314.0,354.0,Arizona,Wildcats,ARIZ,Pac-10,NaN,11,3,8,0
4,2004,Arizona State,9,Sun Devils,ASU,Pac-10,NaN,Pac-12,823.0,429.0,370.0,Arizona State,Sun Devils,ASU,Pac-10,NaN,11,8,3,0


In [48]:
# subset the necessary columns and clean up data
playCallSuccess = merge_2[["season", "id", "team_x", "conference_x", "totalPlays", "passingPlays", "rushingPlays", "games", "wins", "losses"]]

# rename column with suffix from merge
playCallSuccess = playCallSuccess.rename(columns = {"team_x": "team",
                                                   "conference_x": "conference"})

# subset data for conferences we want
power5 = ["ACC", "Big Ten", "Big 12", "Pac-10", "Pac-12", "SEC"]
playCallSuccess = playCallSuccess[playCallSuccess["conference"].isin(power5)]

# drop NA values
playCallSuccess = playCallSuccess.dropna()

# preview the dataset
playCallSuccess.head(10)

,season,id,team,conference,totalPlays,passingPlays,rushingPlays,games,wins,losses
2,2004,333,Alabama,SEC,677.0,224.0,434.0,11,6,5
3,2004,12,Arizona,Pac-10,681.0,314.0,354.0,11,3,8
4,2004,9,Arizona State,Pac-10,823.0,429.0,370.0,11,8,3
5,2004,8,Arkansas,SEC,699.0,279.0,406.0,11,5,6
8,2004,2,Auburn,SEC,753.0,279.0,452.0,12,12,0
10,2004,239,Baylor,Big 12,720.0,385.0,319.0,11,3,8
16,2004,25,California,Pac-10,790.0,308.0,453.0,11,10,1
19,2004,228,Clemson,ACC,639.0,328.0,296.0,11,6,5
20,2004,38,Colorado,Big 12,777.0,379.0,378.0,12,7,5
23,2004,150,Duke,ACC,617.0,284.0,317.0,11,2,9


In [70]:
# identifying quartile ranges for each season to check for outliers
lowOutliers = pd.DataFrame(columns = ["season", "team", "totalPlays"])
highOutliers = pd.DataFrame(columns = ["season", "team", "totalPlays"])

for season in playCallSuccess["season"].unique():
    currSeason = playCallSuccess["totalPlays"].loc[playCallSuccess["season"] == season]
    quartiles = currSeason.quantile([.25,.5,.75])
    
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)

    # append to a dataframe of team-season-total plays that includes all datapoints below lower_bound
    subset = playCallSuccess[["season", "team", "totalPlays"]].loc[playCallSuccess["season"] == season]
    sznLowOutliers = subset.loc[subset["totalPlays"] <= lower_bound]
    sznHighOutliers = subset.loc[subset["totalPlays"] >= upper_bound]
    lowOutliers = lowOutliers.append(sznLowOutliers)
    highOutliers = highOutliers.append(sznHighOutliers)
    
# preview sets of potential outliers
print(lowOutliers)
print(highOutliers)

     season              team  totalPlays
161    2005      Kansas State       371.0
249    2006            Baylor       473.0
381    2007              Duke       533.0
712    2009  Washington State       646.0
1467   2016           Arizona       666.0
1503   2016          Illinois       703.0
     season          team  totalPlays
551    2008      Oklahoma      1000.0
667    2009  Northwestern       998.0
692    2009         Texas       995.0
693    2009     Texas A&M       995.0
791    2010      Oklahoma      1136.0
814    2010    Texas Tech       992.0
1211   2014       Arizona      1141.0
1356   2015       Clemson      1197.0
1484   2016       Clemson      1191.0
